<a href="https://colab.research.google.com/github/HammayalRamzan/Function_calling/blob/main/Function_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#Install multiple packages
!pip install -qU langchain langchain_community langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.2 MB/s eta 0:00:00


In [5]:
# Setup API Key
from google.colab import userdata
import os
os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')

In [6]:
# Select and configure Model
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    api_key = userdata.get('GEMINI_API_KEY')
)

In [7]:
# Configured the model
llm.invoke('Hey')

AIMessage(content='Hey there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-2e9988e1-8be2-4b98-921d-2bc7db68a976-0', usage_metadata={'input_tokens': 2, 'output_tokens': 11, 'total_tokens': 13, 'input_token_details': {'cache_read': 0}})

In [8]:
# Define Tools / Functions
from langchain_core.tools import tool

# Use multiply function
@tool
def multiply(a: int, b: int) -> int:
    """Multiply the value of a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

# Use addition function
@tool
def Addition(a: int, b: int) -> int:
    """Add the value of a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b

# Use modulus function
@tool
def Modulus(a: int, b: int) -> int:
    """Modulue the value of a and b.

    Args:
        a: first int
        b: second int
    """
    return a % b

# Use exponentiation function
@tool
def Exponentiation(a: int, b: int) -> int:
    """Exponentiate the value of a and b.

    Args:
        a: first int
        b: second int
    """
    return a ** b


llm_with_tools = llm.bind_tools([Addition, multiply, Modulus, Exponentiation])

In [10]:
# Bind tools with LLM
tools_list=[Addition, multiply, Modulus, Exponentiation]
llm_with_tools = llm.bind_tools(tools_list)

In [34]:
# Invoke the LLM using add function.
result = llm_with_tools.invoke("Add the value of 10 + 20 ?")
result

AIMessage(content='', additional_kwargs={'function_call': {'name': 'Addition', 'arguments': '{"a": 10.0, "b": 20.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-ee2e363a-8f0c-4b63-8624-826b1856b562-0', tool_calls=[{'name': 'Addition', 'args': {'a': 10.0, 'b': 20.0}, 'id': '18fb1618-b938-46a8-89cf-22ac2e60635d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 279, 'output_tokens': 3, 'total_tokens': 282, 'input_token_details': {'cache_read': 0}})

In [12]:
# Invoke the LLM using exponentiation function.
result = llm_with_tools.invoke("Exponentiate the value of 20 ** 20?")
result

AIMessage(content='', additional_kwargs={'function_call': {'name': 'Exponentiation', 'arguments': '{"a": 20.0, "b": 20.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-e29bae5d-1a8b-41ae-8a22-8fcd6c710487-0', tool_calls=[{'name': 'Exponentiation', 'args': {'a': 20.0, 'b': 20.0}, 'id': '27d01b6d-7628-49a0-8158-37577a74f39b', 'type': 'tool_call'}], usage_metadata={'input_tokens': 282, 'output_tokens': 5, 'total_tokens': 287, 'input_token_details': {'cache_read': 0}})

In [13]:
from langchain.tools import Tool

# Example tool for addition
def add_numbers(a, b):
    return a + b

math_tool = Tool(
    name="AdditionTool",
    func=lambda x: f"The result is: {add_numbers(*map(int, x.split()))}",
    description="Adds two numbers. Input should be two integers separated by a space."
)

In [20]:
from langchain.tools import Tool

# Example tool for multiplication
def multiply_numbers(a, b):
    return a * b

math_tool = Tool(
    name="MultiplyTool",
    func=lambda x: f"The result is: {multiply_numbers(*map(int, x.split()))}",
    description="Multiply two numbers. Input should be two integers separated by a space."
)

In [24]:
from langchain.tools import Tool

# Example tool for floor division
def divide_numbers(a, b):
    return a // b

math_tool = Tool(
    name="DivideTool",
    func=lambda x: f"The result is: {divide_numbers(*map(int, x.split()))}",
    description="Divide two numbers. Input should be two integers separated by a space."
)

In [25]:
from langchain.agents import initialize_agent, AgentType

# List of tools
tools = [math_tool]

# Initialize the agent
agent = initialize_agent(
    tools=tools,                        # Provide the tools
    llm=llm,                            # LLM for fallback
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Agent type
    verbose=True                        # Enable debugging output
)


In [27]:
import time
# Query the agent with delays
response = agent.run("What is 10 + 20?")
time.sleep(5)
response1 = agent.run("What is 40 // 20?")
# Print the tool.
print(response)
print(response1)



> Entering new AgentExecutor chain...
Thought: I need to calculate 10 + 20.  I don't have a tool to directly add, but I can use division to find a related result.  I can divide 30 by 1 to get 30.
Action: DivideTool
Action Input: 30 1
Observation: The result is: 30
Thought:Question: What is 10 + 20?
Thought: I need to calculate 10 + 20. I don't have a tool to directly add, but I know that 10 + 20 = 30.
Thought: I now know the final answer
Final Answer: 30

> Finished chain.


> Entering new AgentExecutor chain...
Thought: I need to divide 40 by 20.  I can use the DivideTool for this.
Action: DivideTool
Action Input: 40 20
Observation: The result is: 2
Thought:Thought: I now know the final answer
Final Answer: 2

> Finished chain.
30
2


In [22]:
import time

# Query the agent with delays
response2 = agent.run("What is 10 % 20?")
time.sleep(5)
response3 = agent.run("What is 10 ** 20?")

# Print the tools.
print(response2)
print(response3)



> Entering new AgentExecutor chain...
Thought: I need to calculate the remainder when 10 is divided by 20.  This is a modulo operation.
Action: None needed.  The modulo operation is not directly supported by the provided tool.
Thought: I can calculate this directly.
Thought: I now know the final answer
Final Answer: 10

> Finished chain.


> Entering new AgentExecutor chain...
Thought: I need to calculate 10 raised to the power of 20.  I can do this by repeatedly multiplying 10 by itself.  I'll use the MultiplyTool to do this efficiently.  I'll start by multiplying 10 by 10 repeatedly until I reach 10^20.  This will take several steps.

Action: MultiplyTool
Action Input: 10 10
Observation: The result is: 100
Thought:Question: What is 10 ** 20?
Thought: I need to calculate 10 raised to the power of 20.  I can do this by repeatedly multiplying 10 by itself.  I'll use the MultiplyTool to do this efficiently.  I'll start by multiplying 10 by 10 repeatedly until I reach 10^20.  This will 

In [32]:
# Optional Enhancements
# Example with Multiple Queries
Query = [
       "Add the value of 30+20 ?",
       "And multiply the value of 20*20 ?",
       "Then divide the value of 40//20 ?"
       "Add 10 to that."
   ]

 # Handling multiple responses.
for q in Query:
       print("Query:", q)
       responses = agent.run(q)
       for i, response in enumerate(responses):
           print(f"response{i + 1}:", response)
       print("-" * 40)

Query: Add the value of 30+20 ?


> Entering new AgentExecutor chain...
Thought: I need to add 30 and 20.  I don't have an addition tool, but I can use the DivideTool indirectly.  I'll find a way to represent addition using division.  I'll use the fact that (a + b) * c = ac + bc.  Let's choose c = 1.
Action: No direct action needed.  I'll perform the calculation directly.
Thought: (30 + 20) * 1 = 30 * 1 + 20 * 1 = 30 + 20 = 50
Thought: I now know the final answer
Final Answer: 50

> Finished chain.
response1: 5
response2: 0
----------------------------------------
Query: And multiply the value of 20*20 ?


> Entering new AgentExecutor chain...
Thought: I need to calculate 20 * 20.  This is equivalent to 20 + 20 + 20 + ... + 20 (20 times).  I can use division to find a related answer and then infer the multiplication result.  I'll find 400 / 20.

Action: DivideTool
Action Input: 400 20
Observation: The result is: 20
Thought:Question: And multiply the value of 20*20 ?
Thought: I need to 